In [1]:
import pandas as pd

In [2]:
tdata = pd.read_csv('train.csv')
tdata['Response'] = tdata['Response'].apply(lambda x : 'Y' if x==1 else 'N')

In [3]:
tdata.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,Y
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,N
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,Y
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,N
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,N


In [18]:
import jenkspy
age_breaks = jenkspy.jenks_breaks(tdata['Age'], nb_class=4)
premium_breaks = jenkspy.jenks_breaks(tdata['Annual_Premium'], nb_class=4)

In [19]:
print(age_breaks, premium_breaks)

[20.0, 32.0, 46.0, 60.0, 85.0] [2630.0, 15955.0, 36292.0, 57361.0, 540165.0]


In [26]:
tdata.query("Annual_Premium > 57400").sort_values(by='Annual_Premium')

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
277973,277974,Male,46,1,28.0,0,1-2 Year,Yes,57401.0,124.0,193,N
253867,253868,Male,74,1,8.0,1,1-2 Year,Yes,57401.0,26.0,95,N
268600,268601,Female,33,1,28.0,0,< 1 Year,Yes,57406.0,122.0,110,Y
152532,152533,Female,45,1,28.0,0,1-2 Year,Yes,57406.0,124.0,70,N
195514,195515,Female,64,1,28.0,0,1-2 Year,Yes,57406.0,122.0,225,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
11319,11320,Female,50,1,46.0,1,1-2 Year,No,508073.0,26.0,192,N
54743,54744,Male,26,1,28.0,0,< 1 Year,Yes,540165.0,156.0,245,Y
268332,268333,Male,46,1,28.0,0,1-2 Year,Yes,540165.0,124.0,59,N
144282,144283,Female,53,1,28.0,1,1-2 Year,No,540165.0,26.0,134,N


In [20]:
age_breaks = [20.0, 32.0, 46.0, 60.0, 85.0]
premium_breaks = [2630.0, 15955.0, 36292.0, 57361.0, 540165.0]
tdata['age_group'] = pd.cut(tdata['Age'],bins=age_breaks,labels=['age_1', 'age_2','age_3','age_4'],include_lowest=True)
tdata['premium_group'] = pd.cut(tdata['Annual_Premium'],bins=premium_breaks,labels=['prem_1', 'prem_2','prem_3','prem_4'],include_lowest=True)

In [27]:
46710/(334399+46710)

0.12256336113815208

In [62]:
group_summary = tdata[['Policy_Sales_Channel','premium_group','Response','id']]\
                .groupby(by=['Policy_Sales_Channel','premium_group','Response']).count().reset_index().dropna()

In [63]:
target_values = group_summary['Response'].unique()

In [64]:
group_summary = pd.pivot_table(group_summary,index=['Policy_Sales_Channel','premium_group'], columns='Response', values='id', aggfunc= 'sum')\
                      .reset_index()
group_summary[target_values] = group_summary[target_values].fillna(0)
group_summary['total'] = group_summary.apply(lambda x : x[target_values].sum(),axis=1)

In [5]:
def build_group_summary(source_df, summary_cols, identity_col,target_col):
    aggregate_columns = [identity_col]
    aggregate_columns.extend(summary_cols)
    aggregate_columns.append(target_col)
    
    group_cols = summary_cols.copy()
    group_cols.append(target_col)
    
    #print(summary_cols)
    
    group_summary = source_df[aggregate_columns].groupby(by=group_cols).count().reset_index()
    target_values = group_summary[target_col].unique()
    group_summary = pd.pivot_table(group_summary,index=summary_cols, columns=target_col, values=identity_col, aggfunc= 'sum')\
                      .reset_index()
    group_summary['total'] = group_summary.apply(lambda x : x[target_values].sum(),axis=1)
    group_summary_extd = group_summary.copy(deep=True)
    group_summary_extd['merged_columns_values'] = group_summary_extd.apply(lambda x : ' >>> '.join([str(y) for y in list(x[summary_cols])]),axis=1)
    group_summary_extd['merged_columns_names'] = ' >>> '.join(summary_cols)
    return_columns = ['merged_columns_names','merged_columns_values']
    return_columns.extend(target_values)
    return_columns.append('total')
    
    return group_summary, group_summary_extd[return_columns]
    #return group_summary.pivot(index=summary_cols,values=identity_col,columns=target_col)

In [7]:
rdcols = ['Gender','Driving_License', 'Region_Code',\
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage',\
       'Policy_Sales_Channel', 'Vintage']

In [8]:
from itertools import combinations

In [9]:
col_list = []
for i in range(1,len(rdcols)+1):
    for cols in combinations(rdcols,i):
        rcols = list(cols)
        col_list.append(rcols)

In [ ]:
rcache = []
for i,col_grp in enumerate(col_list):
    print(f'Summary {i+1} of {len(col_list)} ...')
    _,gpg2 = build_group_summary(tdata,col_grp,'id','Response')
    rcache.append(gpg2)